# Milestone 3 (Task 4)

This notebook is part of Milestone 3, where we use PySpark's MLlib to tune some hyperparameters of a Random Forest to find the optimum model. We use Amazon EMR to achieve this.

In [1]:
# Production mode flag
# When set to False, we only use the first 100 rows to train the model.
# When set to True, we run for the whole dataset.
is_production_mode = True

VBox()

Starting Spark application


Exception in thread cell_monitor-1:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1681324379993_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Bucket name, where to read the source CSV file
s3_bucket_name = 'mds-s3-5-xxx'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Install Required Packages

In [3]:
# Required packages
sc.install_pypi_package("pandas")
sc.install_pypi_package("s3fs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ----------
aiobotocore                2.4.2
aiohttp                    3.8.4
aioitertools               0.11.0
aiosignal                  1.3.1
async-timeout              4.0.2
asynctest                  0.13.0
attrs                      22.2.0
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
botocore                   1.27.59
charset-normalizer         3.1.0
click                      8.1.3
docutils                   0.14
frozenlist                 1.3.3
fsspec                     2023.1.0
idna                       3.4
jmespath                   1.0.1
joblib                     1.2.0
lockfile                   0.11.0
lxml                       4.9.2
multidict                  6.0.4
mysqlclient                1.4.2
nltk                       3.8
nose                       1.3.4
numpy                      1.20.0
pandas                     1.3.5
pip                        20.2.

### Imports

In [5]:
from pyspark.ml import Pipeline
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler, UnivariateFeatureSelector
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read the Data

In [6]:
target_column = "Observed"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
pandas_df = pd.read_csv(f"s3://{s3_bucket_name}/output/ml_data_SYD.csv",
                        index_col=0,
                        parse_dates=True)

if not is_production_mode:
    pandas_df = pandas_df.iloc[:100]

pandas_df = pandas_df.dropna()

feature_cols = list(pandas_df.drop(columns=target_column).columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
pandas_df.shape

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(45989, 26)

In [9]:
feature_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CMCC-CM2-HR4', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CanESM5', 'EC-Earth3-Veg-LR', 'FGOALS-g3', 'GFDL-CM4', 'INM-CM4-8', 'INM-CM5-0', 'KIOST-ESM', 'MIROC6', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1']

### Prepare the Data for ML

Here, we are converting the Pandas data frame into a Spark dataframe.

In [10]:
# Load dataframe and coerce features into a single column called "Features"
# This is a requirement of MLlib
# See more: https://blog.knoldus.com/spark-createdataframe-vs-todf/
training = spark.createDataFrame(pandas_df)
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
training = assembler.transform(training).select("features", target_column)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Find the Best Hyperparameter Settings

With reference to the [official documentation of MLlib for Random Forest](https://spark.apache.org/docs/3.3.1/ml-classification-regression.html#random-forest-regression), we seek the best hyperparameter in the given space.

Additional references:
- [Sparkitecture: Random Forest](https://www.sparkitecture.io/machine-learning/regression/random-forest)
- [Silectis: Random Forest Models With Python and Spark ML](https://www.silect.is/blog/random-forest-models-in-spark-ml/)
- [Pipeline: Machine Learning with PySpark](https://projector-video-pdf-converter.datacamp.com/14989/chapter4.pdf)

In [11]:
rf = RandomForestRegressor(labelCol=target_column)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
rf_parameters = ((ParamGridBuilder()
                  .addGrid(rf.numTrees, [10, 50, 100])
                  .addGrid(rf.maxDepth, [5, 10])
                  .addGrid(rf.bootstrap, [False, True])
                 ).build())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
rf_evaluator = RegressionEvaluator(labelCol=target_column)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
rf_cv = CrossValidator(estimator=rf,
                       estimatorParamMaps=rf_parameters,
                       evaluator=rf_evaluator,
                       numFolds=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
model = rf_cv.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Print the Model Hyperparameters and Score

In [16]:
rf_prediction = model.transform(training)
rf_evaluator.evaluate(rf_prediction)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8.067981500702174

In [17]:
print("\nBest model")
print("\n==========")
print("\n")
print(f"CV Score: {min(model.avgMetrics):.2f}")
print(f"numTrees: {model.bestModel.getNumTrees}")
print(f"maxDepth: {model.bestModel.getMaxDepth()}")
print(f"bootstrap: {model.bestModel.getBootstrap()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Best model



CV Score: 8.18
numTrees: 100
maxDepth: 5
bootstrap: True